# 1. Loading and transforming data

In [5]:
import os
import pandas as pd
import glob

In [33]:
path = "../Data collection/Historic data/raw data/"
files = glob.glob(path + "/*.csv") 
Datasets = pd.DataFrame()
content = [pd.read_csv(filename, index_col=None) for filename in files]
Datasets = pd.concat(content)

Datasets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175954 entries, 0 to 4859
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   titre    175954 non-null  object
 1   date     175954 non-null  object
 2   extrait  175723 non-null  object
dtypes: object(3)
memory usage: 5.4+ MB


In [34]:
import dateparser

In [35]:
def format_date(date):
    return  dateparser.parse(str(date)).date()
Datasets["date"] = Datasets["date"].apply(format_date)

c:\Users\hp\PycharmProjects\pythonProject1\venv\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [ ]:
Datasets.to_csv(r"C:\Users\hp\Desktop\PE\Processing\Outpout\Datasets.csv", index=False)

In [52]:
formated_data=pd.read_csv(r"C:\Users\hp\Desktop\PE\Processing\Outpout\Datasets.csv")
formated_data.sample(5)

,titre,date,extrait
129915,Vidéo. Université d’été CGEM. Jean-Louis Borlo...,2019-09-13,"Dans une déclaration devant Le360, en marge de..."
60829,Air France relance ses vols vers trois villes...,2020-08-18,Air France relance ses vols vers trois villes ...
160768,Première rencontre entre les opérateurs du tou...,2021-11-02,"\nLa ministre du Tourisme a reçu, ce mardi 2 n..."
170385,Maroc-Sénégal : Signature de sept accords et c...,2013-07-26,"\nLe Roi Mohammed VI, accompagné du Prince Mou..."
107513,"""Il n’y a pas encore de découvertes de pétrole...",2014-03-14,"""Il n’y a pas encore de découvertes de pétrole..."


In [53]:
# convert date column into datetima
formated_data['date'] = pd.to_datetime(formated_data['date'], errors = 'coerce')

# delete rows with null values
formated_data.dropna(inplace=True)
formated_data.isna().sum()

titre      0
date       0
extrait    0
dtype: int64

In [56]:
formated_data.to_csv(r"Outpout\Datasets.csv", index=False)

# 2. processing

In [58]:
clean = pd.read_csv(r"C:\Users\hp\Desktop\PE\Processing\Outpout\Datasets.csv", parse_dates=['date'])
clean.date.min()

,titre,date,extrait
0,"Transport – Royal Air Maroc, première compagni...",2022-05-30,"Le transporteur national, Royal Air Maroc (RAM..."
1,Innovation – Maroc: 10 lauréats désignés lors ...,2022-05-28,"Au Maroc, 10 lauréats ont été désignés lors de..."
2,Classement – Tanger Med en 6è position dans la...,2022-05-28,Le Port de Tanger Med a été classé en 6è posit...
3,"Logistics – Après deux années d’absence, Logis...",2022-05-28,La 9ème édition du Salon international du tran...
4,Coopération – Treize MoU d’entente portant sur...,2022-05-26,Les travaux du Forum « Morocco-Israel: Connect...


In [61]:
clean['period']=clean['date'].dt.to_period('M')
clean.sample(5)

,titre,date,extrait,period
156585,\nCovid-19: Une société marocaine se reconvert...,2020-04-21,"Casablanca - \r\n La société marocaine ""...",2020-04
151049,Médecine: le privé déclare la guerre au public,2016-06-22,La guerre entre les médecins du public et du p...,2016-06
7818,L’Office des changes accélère ses process digi...,2021-01-21,Objectif : 95% des demandes d’autorisations vi...,2021-01
81856,Maroc : forte hausse du déficit budgétaire,2019-10-15,Maroc : forte hausse du déficit budgétaire Le ...,2019-10
111171,La CGEM face aux défis et attentes des entrepr...,2019-12-28,La Confédération générale des entreprises du M...,2019-12


In [109]:
grouped_by_M_Y = clean.groupby('period')['extrait'].apply(list)
periods = clean.period.unique().tolist()
extraits_grouped = grouped_by_M_Y.tolist()

In [59]:
corpuses = [ corpus=lower_text(corpus) for corpus in extraits_grouped]

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))

Timestamp('2012-11-22 00:00:00')

### Some text mining functions for processing text.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
np.set_printoptions(precision=2, linewidth=80)
# from nltk import FreqDist
# Gensim
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel

import spacy
# from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
#from bs4 import BeautifulSoup
import unicodedata

from spacy.lang.fr.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer

spacy.load('fr_core_news_md')
import fr_core_news_md #import spacy french stemmer
from sklearn.decomposition import NMF,LatentDirichletAllocation

import pyLDAvis #Nous utilisons pyLDAvis pour créer des visualisations interactives de modèles de sujet.
import pyLDAvis.sklearn

#dealing with date
import dateparser

#output French accents correctly
def convert_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')


#convertisse les documents en minuscule
def lower_text(corpus):
    LowerCorpus=[]
    for doc in corpus:
        lowerDoc=str(doc).lower() #convertissent le texte en minuscules
        lowerDoc=convert_accents(lowerDoc).decode("utf-8") #supprimes les accents
        LowerCorpus.append(lowerDoc)
    return LowerCorpus


def remove_characters(corpus,keep_apostrophes=True):
    filtered_corpus=[]
    for doc in corpus:
        doc = doc.strip()
        if keep_apostrophes:
            doc =re.sub('(https|http)\S*\s?', '',doc) #supprimes les urls
            PATTERN = r'[?|$|&|*|%|@|(|)|~|\d]'
            filtered_doc = re.sub(PATTERN, r'', doc)
            filtered_corpus.append(filtered_doc)
        else:
            PATTERN = r'[^a-zA-Z ]'
            #supprimes les urls
            doc =re.sub('(https|http)\S*\s?', '',doc) #supprimes les urls
            filtered_doc = re.sub(PATTERN, r'', doc)
        
            filtered_corpus.append(filtered_doc)
    return filtered_corpus


#Tokenization
def tokenize_text(corpus):
    tokensCorpus=[]
    for doc in corpus:
        doc_tokens = word_tokenize(doc)
        tokensCorpus.append(doc_tokens)
    return tokensCorpus


#recuperer les mots qui apparaissent dans plusieurs documents
def get_mostCommonWords(corpus,max_freq=100):
    vocabulaire=dict() #dictionnaire qui va contenir le nombre d'occurence des mots dans les documents
    for doc in corpus:
        for word in set(doc.split()): #recupere les mots unique de chaque documents
            if word in vocabulaire:
                vocabulaire[word]+=1
            else:
                vocabulaire[word]=1
    
    #recupere les dont le nombre d'occurences dans les documents > max_freq
    mostCommonsWord=[word for word,value in vocabulaire.items() if value>max_freq ]
        
    return mostCommonsWord


# removing stopwords
def remove_stopwords(corpus,mostCommonsWord):
    filtered_corpus=[]
    for tokens in corpus:
        others_sw=["maroc","morocco","marocain","marocaine","marocains","marocaines","maghreb","météorologique","journée",
                   "méteo","retweet","newspic","twitter","com","pic","newspic","illustration"]
        
        #french_sw = stopwords.words('french') 
        french_sw=list(STOP_WORDS) #get french stopwords
        french_sw.extend(others_sw)
        french_sw.extend(mostCommonsWord)
        
        filtered_tokens = [token for token in tokens.split() if token not in french_sw and len(token)>2]
        filtred_text=' '.join(filtered_tokens) #reforme le text du documents separé par espace
        filtered_corpus.append(filtred_text)
    return filtered_corpus


#lemmatisation
def lemm_tokens(corpus):
    
    nlp = fr_core_news_md.load() #initialisation du model "fr_core_news_md" de spacy
    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    corpus_lemms=[]
    
    idx_doc=[] #liste qui va stocker les indices documents qui seront dans le corpus
    for idx,document in enumerate(corpus):
        doc = nlp(document)
        lemms=[token.lemma_ for token in doc if token.pos_ in allowed_postags] #recupere les lemms des tokens
        
        if len(lemms)>5: #supprime les document qui ne contient pas plus de 2 mots
            text=' '.join(lemms) #reforme le text du documents separé par espace
            corpus_lemms.append(text)
            idx_doc.append(idx) #ajoute l'indice du documents
            
    return corpus_lemms,idx_doc


#fonction qui supprimes les documents vides ou tres courte
def remove_shortDocument(corpus,min_length=3):
    filtred_corpus=[]
    idx_doc=[]
    for idx,doc in enumerate(corpus):
        
        if len(doc.split())>min_length:
            filtred_corpus.append(doc)
            idx_doc.append(idx)
        
    
    return filtred_corpus,idx_doc


def make_bigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=2, threshold=10) # higher threshold fewer phrases
    # Un moyen plus rapide d'obtenir une phrase matraquée comme un trigramme / bigramme
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    corpus_bigram=[" ".join(bigram_mod[doc]) for doc in texts]
    return corpus_bigram


def preprocessing(corpus):
    
    corpus=lower_text(corpus)
    corpus=remove_characters(corpus)
    corpus=tokenize_text(corpus)
    #corpus=remove_mostCommonWords(corpus,max_freq=20)
    corpus=remove_stopwords(corpus)
    corpus,idx_docs=lemm_tokens(corpus)
    
    
    return corpus,idx_docs
